In [2]:
pip install transformers datasets torch

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install --upgrade transformers

     ---------------------------------------- 10.1/10.1 MB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 2.4/2.4 MB 3.7 MB/s eta 0:00:00
     -------------------------------------- 303.8/303.8 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.11.4
    Uninstalling tokenizers-0.11.4:
      Successfully uninstalled tokenizers-0.11.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.24.0
    Uninstalling transformers-4.24.0:
      Successfully uninstalled transformers-4.24.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import transformers
print(transformers.__version__)  # Should be at least 4.30 or later

4.47.1


In [5]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

# Load the dataset

# Display the first few rows
print(data.head())
data['prompt'] = data['Environmental Factor'].apply(lambda x: f"predict: {x}")
data['output'] = data['Yield (kg)']  # Replace with your target column

   Area (hectares)  Temperature (C)  Rainfall (mm)  Soil Moisture (%)  \
0         4.058131        22.888505     518.736507          31.134739   
1         9.531786        15.323278     783.378885          51.931728   
2         7.453942        23.900989     499.055925          33.050157   
3         6.187256        25.052929     952.011193          31.975335   
4         1.982177        24.798952     816.487951          29.300240   

     Yield (kg)  Environmental Factor  
0  13759.859008               1.00000  
1  34739.551732               0.52012  
2  30654.719899               1.00000  
3  25815.903795               1.00000  
4   7054.423701               1.00000  


In [4]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(data['prompt'], data['output'], test_size=0.2)

In [6]:
pip install sentencepiece

     -------------------------------------- 991.5/991.5 kB 6.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sentencepiece
print(sentencepiece.__version__)  # This should print the installed version

0.2.0


In [15]:
pip install transformers[torch]

     -------------------------------------- 336.4/336.4 kB 6.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Generate prompt and output columns
import pandas as pd
data = pd.read_csv('synthetic_apple_data.csv')
data['prompt'] = data.apply(lambda row: f"predict: Temperature {row['Temperature (C)']}C, Rainfall {row['Rainfall (mm)']}mm, Soil Moisture {row['Soil Moisture (%)']}%", axis=1)
data['output'] = data['Yield (kg)'].astype(str)  # Ensure output is a string

# Split data for training
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(data['prompt'], data['output'], test_size=0.2)

# Tokenize the data
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)
import torch

class AppleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = tokenizer(list(labels), truncation=True, padding=True).input_ids

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx]),
        }

train_dataset = AppleDataset(train_encodings, train_labels)
val_dataset = AppleDataset(val_encodings, val_labels)
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained('t5-small')

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
C:\Users\91781\anaconda3\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,4.835400,4.260203
2,4.010700,3.787669
3,3.815200,3.718032


TrainOutput(global_step=600, training_loss=4.949269886016846, metrics={'train_runtime': 8547.5258, 'train_samples_per_second': 0.281, 'train_steps_per_second': 0.07, 'total_flos': 29183075942400.0, 'train_loss': 4.949269886016846, 'epoch': 3.0})

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Define features and target variable
file_path='synthetic_apple_data.csv'
apple_data=pd.read_csv(file_path)
features = apple_data[['Area (hectares)', 'Temperature (C)', 'Rainfall (mm)', 'Soil Moisture (%)']]
target = apple_data['Yield (kg)']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Mock predictions (Replace this with predictions from your T5 model)
# Example: Convert the test data to text format, pass it through T5, and parse predictions back into numeric form
y_pred = y_test * 0.95  # Simulating predictions with 95% accuracy for illustration

# Calculate regression metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Display metrics
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R² Score: {r2}")


from sklearn.metrics import accuracy_score, classification_report

# Define a classification rule (e.g., high yield > 20,000 kg)
threshold = 20000
y_test_class = ['high' if y > threshold else 'low' for y in y_test]
y_pred_class = ['high' if y > threshold * 0.95 else 'low' for y in y_pred]  # Simulating predictions

# Calculate accuracy
accuracy = accuracy_score(y_test_class, y_pred_class)

# Display results
print(f"Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:\n", classification_report(y_test_class, y_pred_class))


MAE: 1120.2940509114896
MSE: 1729794.5403744623
RMSE: 1315.216537447147
R² Score: 0.9908907511623631
Accuracy: 100.00%

Classification Report:
               precision    recall  f1-score   support

        high       1.00      1.00      1.00       110
         low       1.00      1.00      1.00        90

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

